# <h1 align="center"><font color="red">Saídas estruturadas em JSON</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']


from openai import OpenAI
client = OpenAI(api_key=Eddy_key_openai)

from pydantic import BaseModel

In [2]:
EGCI = """
Eddy Giusepe Chirinos Isidro Nasceu em Perú, mas atualmente mora em Brasil. 
Ele é Senior Data Scientist e atualmente trabalha na Tractian. 
"""

In [7]:
json_schema = {
    "name": "NamedEntities",
    "schema": {
                "type": "object",
                "properties": {"entities": {"type": "array",
                                            "description": "Lista de nomes de entidades e seus tipos correspondentes",
                                            "items": {"type": "object",
                                                      "properties": {"name": {"type": "string",
                                                                              "description": "O nome real conforme especificado no texto, por exemplo, o nome de uma pessoa ou o nome do país"
                                                                             },
                                                                     "type": {"type": "string",
                                                                              "description": "O tipo de entidade, como 'Pessoa' ou 'Organização'",
                                                                              "enum": ["Person", "Organization", "Location", "DateTime"]
                                                                             }
                                                                    },
                                                      "required": ["name", "type"],
                                                      "additionalProperties": False
                                                     }
                                           }
                              },
                "required": ["entities"],
                "additionalProperties": False
              },
    "strict": True
}

print(json_schema)

{'name': 'NamedEntities', 'schema': {'type': 'object', 'properties': {'entities': {'type': 'array', 'description': 'Lista de nomes de entidades e seus tipos correspondentes', 'items': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'O nome real conforme especificado no texto, por exemplo, o nome de uma pessoa ou o nome do país'}, 'type': {'type': 'string', 'description': "O tipo de entidade, como 'Pessoa' ou 'Organização'", 'enum': ['Person', 'Organization', 'Location', 'DateTime']}}, 'required': ['name', 'type'], 'additionalProperties': False}}}, 'required': ['entities'], 'additionalProperties': False}, 'strict': True}


In [8]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": """Você é um sistema Named Entity Recognition (NER). Seu trabalho é identificar e retornar
                          todos os nomes de entidades e seus tipos para um determinado pedaço de texto. Você deve
                          obedecer estritamente apenas aos seguintes tipos de entidade: Person, Location, Organization
                          e DateTime. Se não tiver certeza sobre o tipo de entidade, ignore-o.
                       """,
        },
        {
            "role": "user",
            "content": EGCI
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": json_schema,
    }
)


In [9]:

print(completion.choices[0].message.content)

{"entities":[{"name":"Eddy Giusepe Chirinos Isidro","type":"Person"},{"name":"Perú","type":"Location"},{"name":"Brasil","type":"Location"},{"name":"Tractian","type":"Organization"}]}


In [10]:
import pprint
import json



pprint.pprint(json.loads(completion.choices[0].message.content), indent=4)

{   'entities': [   {'name': 'Eddy Giusepe Chirinos Isidro', 'type': 'Person'},
                    {'name': 'Perú', 'type': 'Location'},
                    {'name': 'Brasil', 'type': 'Location'},
                    {'name': 'Tractian', 'type': 'Organization'}]}
